In [1]:
from datasets import load_dataset, concatenate_datasets
import pandas as pd
from sklearn.utils import resample
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate
from optimum.bettertransformer import BetterTransformer
import torch
from pynvml import *

c:\Users\Gusso\anaconda3\envs\hugging_torch_2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("Torch version:",torch.__version__)

print("Is CUDA enabled?",torch.cuda.is_available())

torch.cuda.empty_cache()

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()


Torch version: 2.1.1
Is CUDA enabled? True


In [3]:
dataset = load_dataset("financial_phrasebank", 'sentences_50agree')

neutrals = dataset.filter(lambda example: example["label"] == 1)
positives = dataset.filter(lambda example: example["label"] == 2)
negatives = dataset.filter(lambda example: example["label"] == 0)

short_positives = positives['train'].select(range(len(negatives['train'])))

print(short_positives)

downsized_dataset = concatenate_datasets([short_positives, negatives['train']])

print(downsized_dataset)

Found cached dataset financial_phrasebank (C:/Users/Gusso/.cache/huggingface/datasets/financial_phrasebank/sentences_50agree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141)
100%|██████████| 1/1 [00:00<00:00, 71.44it/s]
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\financial_phrasebank\sentences_50agree\1.0.0\550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141\cache-4069f8bdc2072a81.arrow
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\financial_phrasebank\sentences_50agree\1.0.0\550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141\cache-255302c37e6b94c9.arrow
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\financial_phrasebank\sentences_50agree\1.0.0\550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141\cache-913aca77f5bc0114.arrow


Dataset({
    features: ['sentence', 'label'],
    num_rows: 604
})
Dataset({
    features: ['sentence', 'label'],
    num_rows: 1208
})


In [4]:
def replace_values(example):
    # Replace values in the label column
    if "label" in example:
        example["label"] = 1 if example["label"] == 2 else example["label"]
    return example

# Use the map function to apply the custom mapping
downsized_dataset = downsized_dataset.map(replace_values)
downsized_dataset = downsized_dataset.train_test_split(test_size=0.1, shuffle=True)
print(downsized_dataset)

Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\financial_phrasebank\sentences_50agree\1.0.0\550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141\cache-9ab259d81c7af6d1.arrow


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 1087
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 121
    })
})


In [5]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding='max_length', truncation=True)

tokenized_datasets = downsized_dataset['train'].map(tokenize_function, batched=True)
tokenized_datasets_eval = downsized_dataset['test'].map(tokenize_function, batched=True)

In [6]:
model = AutoModelForSequenceClassification.from_pretrained("bert-large-uncased", load_in_8bit=True)

# model = model.to(0)

print(model.get_memory_footprint())

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin c:\Users\Gusso\anaconda3\envs\hugging_torch_2\lib\site-packages\bitsandbytes\libbitsandbytes_cuda117.dll
CUDA SETUP: CUDA runtime path found: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.7\bin\cudart64_110.dll
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary c:\Users\Gusso\anaconda3\envs\hugging_torch_2\lib\site-packages\bitsandbytes\libbitsandbytes_cuda117.dll...


c:\Users\Gusso\anaconda3\envs\hugging_torch_2\lib\site-packages\bitsandbytes\cuda_setup\main.py:152: UserWarning: C:\Users\Gusso\anaconda3\envs\hugging_torch_2 did not contain ['cudart64_110.dll', 'cudart64_120.dll'] as expected! Searching further paths...
  warn(msg)
c:\Users\Gusso\anaconda3\envs\hugging_torch_2\lib\site-packages\bitsandbytes\cuda_setup\main.py:152: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('C:/Users/Gusso/anaconda3/envs/hugging_torch_2/Library/usr/bin'), WindowsPath('C:/ProgramData/DockerDesktop/version-bin'), WindowsPath('C:/Users/Gusso/anaconda3/envs/hugging_torch_2/Library/mingw-w64/bin'), WindowsPath('C:/Users/Gusso/anaconda3/Library/usr/bin')}
  warn(msg)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task 

367257604


In [7]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets_eval,
    compute_metrics=compute_metrics,
)


In [8]:
result = trainer.train()
print_summary(result)

  0%|          | 0/408 [00:00<?, ?it/s]

                                                 
 33%|███▎      | 136/408 [01:31<02:41,  1.68it/s]

{'eval_loss': nan, 'eval_accuracy': 0.4628099173553719, 'eval_runtime': 4.574, 'eval_samples_per_second': 26.454, 'eval_steps_per_second': 3.498, 'epoch': 1.0}


                                                 
 67%|██████▋   | 272/408 [03:00<01:19,  1.71it/s]

{'eval_loss': nan, 'eval_accuracy': 0.4628099173553719, 'eval_runtime': 4.5825, 'eval_samples_per_second': 26.405, 'eval_steps_per_second': 3.492, 'epoch': 2.0}


                                                 
100%|██████████| 408/408 [04:28<00:00,  1.52it/s]


{'eval_loss': nan, 'eval_accuracy': 0.4628099173553719, 'eval_runtime': 4.4955, 'eval_samples_per_second': 26.916, 'eval_steps_per_second': 3.559, 'epoch': 3.0}
{'train_runtime': 268.615, 'train_samples_per_second': 12.14, 'train_steps_per_second': 1.519, 'train_loss': 0.3140001483992034, 'epoch': 3.0}
Time: 268.62
Samples/second: 12.14


NVMLError_LibraryNotFound: NVML Shared Library Not Found

In [ ]:
##model = BetterTransformer.transform(model)

trainer2 = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets_eval,
    compute_metrics=compute_metrics,
)

In [ ]:
validation_dataset = load_dataset('zeroshot/twitter-financial-news-sentiment')

def filter_function(example):
    return example["label"] != 2

# Use the filter method to remove records
filtered_dataset = validation_dataset.filter(filter_function)

def tokenize_function2(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True)

tokenized_datasets_validation = filtered_dataset.map(tokenize_function2, batched=True)

# Print the modified dataset
print(dataset)

Found cached dataset csv (C:/Users/Gusso/.cache/huggingface/datasets/zeroshot___csv/zeroshot--twitter-financial-news-sentiment-ccca0f3c622c5b67/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 2/2 [00:00<00:00, 84.82it/s]
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\zeroshot___csv\zeroshot--twitter-financial-news-sentiment-ccca0f3c622c5b67\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-b2d98c45395bc49c.arrow
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\zeroshot___csv\zeroshot--twitter-financial-news-sentiment-ccca0f3c622c5b67\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-98a6e9e55fa382e5.arrow
Loading cached processed dataset at C:\Users\Gusso\.cache\huggingface\datasets\zeroshot___csv\zeroshot--twitter-financial-news-sentiment-ccca0f3c622c5b67\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 4846
    })
})


In [ ]:
trainer2.evaluate(eval_dataset=tokenized_datasets_validation['validation'])

100%|██████████| 411/411 [00:52<00:00,  7.83it/s]


{'eval_loss': nan,
 'eval_accuracy': 0.4221411192214112,
 'eval_runtime': 52.7567,
 'eval_samples_per_second': 15.581,
 'eval_steps_per_second': 7.79}